In [48]:
%%bash
rm -r ./data
rm -r ./code

In [49]:
%%bash
wget -s -q -O - https://dvc.org/s3/examples/so/code.tgz | tar -xzvf -
mkdir -p data
wget -s -P data https://dvc.org/s3/so/25K/Posts.xml.tgz
mkdir -p svc

x code/
x code/requirements.txt
x code/train_model.py
x code/split_train_test.py
x code/xml_to_tsv.py
x code/evaluate.py
x code/featurization.py
--2018-09-22 15:27:09--  https://dvc.org/s3/so/25K/Posts.xml.tgz
Resolving dvc.org... 52.3.53.115, 52.55.191.55, 52.23.126.223, ...
Connecting to dvc.org|52.3.53.115|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://s3-us-west-2.amazonaws.com/dvc-share/so/25K/Posts.xml.tgz [following]
--2018-09-22 15:27:10--  https://s3-us-west-2.amazonaws.com/dvc-share/so/25K/Posts.xml.tgz
Resolving s3-us-west-2.amazonaws.com... 52.218.209.176
Connecting to s3-us-west-2.amazonaws.com|52.218.209.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10576022 (10M) [text/xml]
Saving to: ‘data/Posts.xml.tgz’

     0K .......... .......... .......... .......... ..........  0%  141K 73s
    50K .......... .......... .......... .......... ..........  0%  266K 56s
   100K .......... .......... .

In [50]:
! dvc init    

Error: Failed to initiate dvc: '.dvc' exists. Use '-f' to force.


In [51]:
! dvc add dev/data/Posts.xml.tgz

Adding 'data/Posts.xml.tgz' to 'data/.gitignore'.
Saving 'data/Posts.xml.tgz' to cache '.dvc/cache'.
Created 'reflink': .dvc/cache/2f/412200dc53fb97dcac0353b609d199 -> data/Posts.xml.tgz
Saving information to 'data/Posts.xml.tgz.dvc'.


In [64]:
%%bash
dvc run -d dev/data/Posts.xml.tgz \
              -o dev/data/Posts.xml \
              -f dvc/extract.dvc \
              tar -xvf data/Posts.xml.tgz -C data

Running command:
	tar -xvf data/Posts.xml.tgz -C data
Saving 'data/Posts.xml' to cache '.dvc/cache'.
Created 'reflink': .dvc/cache/a3/04afb96060aad90176268345e10355 -> data/Posts.xml
Saving information to 'dvc/extract.dvc'.


x Posts.xml


In [65]:
%%bash
dvc run -d code/xml_to_tsv.py -d data/Posts.xml \
              -o data/Posts.tsv \
              -f dvc/prepare.dvc \
              python code/xml_to_tsv.py data/Posts.xml data/Posts.tsv

Running command:
	python code/xml_to_tsv.py data/Posts.xml data/Posts.tsv
Saving 'data/Posts.tsv' to cache '.dvc/cache'.
Created 'reflink': .dvc/cache/66/5f20ef829fc3664eff4d71952d5b87 -> data/Posts.tsv
Saving information to 'dvc/prepare.dvc'.


In [66]:
%%writefile code/split_train_test.py
import sys
import pandas as pd
from sklearn.model_selection import train_test_split

try:  # python2
    reload(sys)
    sys.setdefaultencoding('utf-8')
except NameError:
    pass


def usage(msg):
    if msg:
        sys.stderr.write('{}\n\n'.format(msg))
    sys.stderr.write('python split_train_test.py data ratio seed train test\n\n')
    sys.stderr.write('\tdata \t input data set to split.\n')
    sys.stderr.write('\tratio \t train set ratio (double). Example: 0.3\n')
    sys.stderr.write('\tseed \t random state (integer). Example: 20170423\n')
    sys.stderr.write('\ttrain \t train dataset output.\n')
    sys.stderr.write('\ttest \t test dataset output.\n')
    sys.exit(1)


if len(sys.argv) != 6:
    usage('Wrong number of arguments. Usage:')


input = sys.argv[1]
test_ratio = float(sys.argv[2])
seed = int(sys.argv[3])
train = sys.argv[4]
test = sys.argv[5]

df = pd.read_csv(
    input,
    encoding='utf-8',
    header=None,
    delimiter='\t',
    names=['id', 'label', 'text']
)

df_positive = df[df['label'] == 1]
df_negative = df[df['label'] == 0]

sys.stderr.write('Positive size {}, negative size {}\n'.format(
    df_positive.shape[0],
    df_negative.shape[0]
))


def sub_df_by_ids(df, ids):
    df_train_order = pd.DataFrame(data={'id': ids})
    return df.merge(df_train_order, on='id')


def train_test_split_df(df, ids, test_ratio, seed):
    train_ids, test_ids = train_test_split(ids, test_size=test_ratio, random_state=seed)
    return sub_df_by_ids(df, train_ids), sub_df_by_ids(df, test_ids)


df_pos_train, df_pos_test = train_test_split_df(df, df_positive.id, test_ratio, seed)
df_neg_train, df_neg_test = train_test_split_df(df, df_negative.id, test_ratio, seed)

df_train = pd.concat([df_pos_train, df_neg_train])
df_test = pd.concat([df_pos_test, df_neg_test])

df_train.to_csv(train, sep='\t', header=False, index=False)
df_test.to_csv(test, sep='\t', header=False, index=False)

Overwriting code/split_train_test.py


In [67]:
%%bash 
dvc run -d code/split_train_test.py -d data/Posts.tsv \
              -o data/Posts-train.tsv -o data/Posts-test.tsv \
              -f dvc/split.dvc \
              python code/split_train_test.py data/Posts.tsv 0.2 20170426 \
                                              data/Posts-train.tsv data/Posts-test.tsv

Running command:
	python code/split_train_test.py data/Posts.tsv 0.2 20170426 data/Posts-train.tsv data/Posts-test.tsv
Saving 'data/Posts-train.tsv' to cache '.dvc/cache'.
Created 'reflink': .dvc/cache/99/16619870195e5b5363931b4fdc551c -> data/Posts-train.tsv
Saving 'data/Posts-test.tsv' to cache '.dvc/cache'.
Created 'reflink': .dvc/cache/df/96975a3e98890294f4ab248784aa5c -> data/Posts-test.tsv
Saving information to 'dvc/split.dvc'.


Positive size 492, negative size 24508


In [68]:
%%bash
dvc run -d code/featurization.py -d data/Posts-train.tsv -d data/Posts-test.tsv \
              -o data/matrix-train.pkl -o data/matrix-test.pkl \
              -f dvc/featurize.dvc \
              python code/featurization.py data/Posts-train.tsv data/Posts-test.tsv \
                                           data/matrix-train.pkl data/matrix-test.pkl

Running command:
	python code/featurization.py data/Posts-train.tsv data/Posts-test.tsv data/matrix-train.pkl data/matrix-test.pkl
Saving 'data/matrix-train.pkl' to cache '.dvc/cache'.
Created 'reflink': .dvc/cache/a3/820b22697b93f080eb9090c55da651 -> data/matrix-train.pkl
Saving 'data/matrix-test.pkl' to cache '.dvc/cache'.
Created 'reflink': .dvc/cache/2e/3d1a7d1e540cd3e6325a86f3cf8ff5 -> data/matrix-test.pkl
Saving information to 'dvc/featurize.dvc'.


The input data frame data/Posts-train.tsv size is (19999, 3)
The output matrix data/matrix-train.pkl size is (19999, 5002) and data type is float64
The input data frame data/Posts-test.tsv size is (5001, 3)
The output matrix data/matrix-test.pkl size is (5001, 5002) and data type is float64


In [69]:
%%bash
dvc run -d code/train_model.py -d data/matrix-train.pkl \
              -o data/model.pkl \
              -f dvc/train.dvc \
              python code/train_model.py data/matrix-train.pkl 20170426 data/model.pkl

Running command:
	python code/train_model.py data/matrix-train.pkl 20170426 data/model.pkl
Saving 'data/model.pkl' to cache '.dvc/cache'.
Created 'reflink': .dvc/cache/ab/4cfa4c28b121d1a9b3dbb33715dcaf -> data/model.pkl
Saving information to 'dvc/train.dvc'.


Input matrix size (19999, 5002)
X matrix size (19999, 5000)
Y matrix size (19999,)


In [70]:
%%bash
dvc run -d code/evaluate.py -d data/model.pkl -d data/matrix-test.pkl \
              -M auc.metric \
              -f dvc/evaluate.dvc \
              python code/evaluate.py data/model.pkl data/matrix-test.pkl auc.metric

Running command:
	python code/evaluate.py data/model.pkl data/matrix-test.pkl auc.metric
Saving information to 'dvc/evaluate.dvc'.


In [75]:
! dvc metrics show -a

Error: Failed to show metrics: No metric files in this repository. Use 'dvc metrics add' to add a metric file to track.


In [72]:
%%bash
ls

README.md
Talk.ipynb
auc.metric
binder
bokeh.ipynb
code
data
dvc
dvc.ipynb
geojson-extension.geojson
img
imports.py
index.ipynb
input
output


In [82]:
!dvc repro -m dvc/evaluate.dvc

Reproducing 'dvc/extract.dvc'
Running command:
	tar -xvf data/Posts.xml.tgz -C data
tar: Error opening archive: Failed to open 'data/Posts.xml.tgz'
Error: Failed to reproduce 'dvc/evaluate.dvc': Failed to reproduce 'dvc/extract.dvc': Stage 'dvc/extract.dvc' cmd tar -xvf data/Posts.xml.tgz -C data failed


In [81]:
!dvc pipeline show dvc/evaluate.dvc

dvc/extract.dvc
dvc/prepare.dvc
dvc/split.dvc
dvc/featurize.dvc
dvc/train.dvc
dvc/evaluate.dvc
